In [1]:
from time import strftime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
import datetime 
import re
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from math import factorial
from scipy.signal import savgol_filter

In [2]:
return_scale = 1
#Autocorrelation lags
lags = 45
lags_min = lags - lags

## VIX Data

In [4]:
%pip install jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 KB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.1/439.1 KB 6.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 36.1 MB/s eta 0:00:0000:0100:01
  Using cached terminado-0.17.0-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━

In [5]:
d1_filename = glob.glob(d1_name)[0]
d1= pd.read_csv(d1_filename)
d1["returns"] = d1["CLOSE"] - d1["CLOSE"].shift(return_scale)
d1=d1.dropna()

NameError: name 'd1_name' is not defined

## All User Data

In [ ]:
partial_name = f'Data/All_Metaculus*'
filename = glob.glob(partial_name)[0]
df= pd.read_csv(filename, parse_dates=['Time'], index_col = ['Time'])
df= df[df['Users'] != 0]
#df=df.loc[start_date:]
X= df.index
print(df)

plt.figure(figsize=(8, 6))
for i in df.columns:
        plt.plot(X,df[i], label = i)

plt.title("Timeseries of All Metaculus Data")
plt.xlabel("Date")
plt.ylabel("User Data")
plt.legend()
plt.show()

In [ ]:
all_daily = df.copy()
all_daily = all_daily.resample("1D").sum()
print(all_daily.head(25))

In [ ]:
all_returns = df.iloc[:,1] - df.iloc[:,1].shift(return_scale)
all_returns = all_returns.dropna().to_frame()


all_returns.columns = ["Returns"]
all_returns.index.rename("DATE", inplace= True)
#all_returns["Returns"] = np.log(all_returns["Returns"])

plt.figure(figsize=(8, 6))
for i in all_returns.columns:
        plt.plot(all_returns.index,all_returns[i], label = i)

plt.title("Timeseries of {} Day Returns on All Metaculus Users".format(return_scale))
plt.xlabel("Time")
plt.ylabel("Returns on User Data")
plt.legend()
plt.show()